<a href="https://colab.research.google.com/github/RamezHatab/Spotify-Suggestion-Model/blob/main/Copy_of_Spotify_Suggestion_Model_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify Suggestion Model V1

## Infrastructure

In [ ]:
import sys
!{sys.executable} -m pip install spotipy

In [ ]:
username = "agwx2"
cid = "17c11cf9271f4980a85e6aab83cba6bb" 
secret = "015c9abf126340a7981e7ba64dbe7828"
good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
bad_playlist_id = "53FPslFSIAGvVyERcNG4J3"
source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from statistics import mean, stdev
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression

## Data Collection

In [ ]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [ ]:
good_playlist = sp.user_playlist(username, good_playlist_id)
good_tracks = good_playlist["tracks"]
good_songs = good_tracks["items"]

while good_tracks['next']:
    good_tracks = sp.next(good_tracks)
    for item in good_tracks["items"]:
        good_songs.append(item)

good_ids = []
for i in range(len(good_songs)):
    good_ids.append(good_songs[i]['track']['id'])
# print(len(good_ids))
# print(good_ids)
print(good_songs)

In [ ]:
bad_playlist = sp.user_playlist(username, bad_playlist_id)
bad_tracks = bad_playlist["tracks"]
bad_songs = bad_tracks["items"]

while bad_tracks['next']:
    bad_tracks = sp.next(bad_tracks)
    for item in bad_tracks["items"]:
        bad_songs.append(item)

bad_ids = []
for i in range(len(bad_songs)):
    bad_ids.append(bad_songs[i]['track']['id'])
print(len(bad_ids))
print(bad_ids)

In [ ]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids)

In [ ]:
training_features = []

for i in range(0, len(good_ids), 50):
    audio_features = sp.audio_features(good_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 1

for i in range(0, len(bad_ids), 50):
    audio_features = sp.audio_features(bad_ids[i : i + 50])
    for track in audio_features:
        training_features.append(track)
        training_features[-1]['target'] = 0

print(len(training_features))
print(training_features)

In [ ]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        source_features.append(track)
        source_features[-1]['target'] = 0  # arbitrary

print(len(source_features))
print(source_features)

## Data Preparation

In [ ]:
trainingData = pd.DataFrame(training_features)

In [ ]:
trainingData

In [ ]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [ ]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
x_test

In [ ]:
y_test

## Model Evaluation

In [ ]:
# linear regression is a good start, but does not work well for classification problems
linreg = LinearRegression()
linreg.fit(x_train, y_train)
linreg_pred = linreg.predict(x_test)

print('Mean squared error: %.2f' % mean_squared_error(y_test, linreg_pred))
linreg_pred

In [ ]:
# k-neighbors algorithm is popular but not so applicable in this case, no local structure to data
knn = KNeighborsClassifier(3)
knn.fit(x_train, y_train)
knn_pred = knn.predict(x_test)

score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, knn_pred))

In [ ]:
# decision tree is a common element in machine learning, useful in classification
dtc = DecisionTreeClassifier(min_samples_split=100)
dtc.fit(x_train, y_train)
dtc_pred = dtc.predict(x_test)

score = accuracy_score(y_test, dtc_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, dtc_pred))

In [ ]:
# random forest combines multiple decision trees and uses bagging
# (training multiple trees on different sections of training data, averaging the result)
rfc = RandomForestClassifier(n_jobs=1, random_state=1)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)

score = accuracy_score(y_test, rfc_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, rfc_pred))

In [ ]:
# ada boost combines multiple decision trees and uses boosting
# (training a tree repeatedly on the result of previous training to reduce error in each subsequent tree)
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
ada_pred = ada.predict(x_test)

score = accuracy_score(y_test, ada_pred) * 100
print("Accuracy using Ada Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, ada_pred))

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=2)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test)

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))

In [ ]:
model = ada

## Generate Predictions

In [ ]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [ ]:
sourceData

In [ ]:
pred = model.predict(sourceData)

In [ ]:
source_playlist_songs = source_playlist['tracks']['items']

print("ACCEPTED")

i = 0
for prediction in pred:
    if prediction == 1:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1


print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0:
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1